# Imports

In [1]:
import pandas as pd
import numpy as np
from bs4 import BeautifulSoup
import requests
from datetime import datetime
import re
from sqlalchemy import create_engine
from sqlalchemy.orm import sessionmaker
import json

from selenium import webdriver
from selenium.webdriver.firefox.options import Options
from selenium.webdriver.common.by import By
from selenium.webdriver.support.wait import WebDriverWait
from selenium.webdriver.support import expected_conditions as EC

In [2]:
# functions

def request_soup(url_link):    
    headers = {'User-Agent' : 'Mozilla/5.0 (Macintosh; Intel Mac OS X x.y; rv:42.0) Gecko/20100101 Firefox/42.0'}    
    page = requests.get( url, headers = headers)
    soup_obj = BeautifulSoup(page.text, 'html.parser')
    return( soup_obj )

def composition_to_df(list_of_comp):
    """ Creates a dataframe from a list of compositions"""
    keys = []
    values = []

    for idx, element in enumerate(list_of_comp):
        # if idx is even, element is a key (column in dataframe)
        if idx % 2 == 0:
            keys.append(element)
        else:
            values.append(element.strip('%,'))

    # final dataframe
    res = dict(zip(keys,values))
    res = pd.DataFrame(res, index = [0])
    return (res)

# Data Requesting

##  Home Page Scraping

In [20]:
# all products url
url = 'https://www2.hm.com/en_us/men/products/jeans.html'

# headers for request
headers = {'User-Agent' : 'Mozilla/5.0 (Macintosh; Intel Mac OS X x.y; rv:42.0) Gecko/20100101 Firefox/42.0'}

# requesting
page = requests.get(url=url, headers=headers)

# instatiating bs4 object
soup = BeautifulSoup(page.text, 'html.parser')

In [21]:
# finding load more products element
p = soup.find('div', class_='load-more-products')

# all products
all_products = int(p.find('h2').get('data-total'))

# products per page
products_per_page = int(p.find('h2').get('data-items-shown'))

# rounding up numer of pages needed for web scraping
total_pages = np.ceil(all_products/products_per_page)


##  All products in Home Page Scraping

In [22]:
# creating a page with all products
url_all_prods = url + '?&offset=0&page-size={}'.format(int(total_pages*products_per_page))

all_prods = requests.get(url = url_all_prods, headers=headers)

In [23]:
soup = BeautifulSoup(all_prods.text, 'html.parser')#.get('li', class_='product-item')

# soup.find('li', class_ = 'product-item').find('a').get('href') #.get('item-link')  #.get('item-link') #, class_ = 'item-link')
# all find all products listed in homepage
products = soup.find_all('li', class_='product-item')

# get link to all projects
home_links = ['https://www2.hm.com' + link.find('a').get('href') for link in products ]

##  All products in Each Product Page

In [24]:
# resulting list of all products to scrap
links = []

for link in home_links:
    # scrap each product in home page list
    single_product = requests.get(link, headers = headers)
    soup = BeautifulSoup(single_product.text, 'html.parser')

    # gets the links to all products listed in a page
    products_ul = soup.find('ul', class_='inputlist clearfix')
    products = products_ul.find_all('a')

    links_ul = []
    links_ul = [ 'https://www2.hm.com' + item.get('href') for item in products]
    links.extend(links_ul)

In [25]:
# getting all unique products listed

# converting to a set and then back to list
links = list(set(links))
links.sort()

In [26]:
# defining base dataframe
df_prods = pd.DataFrame()

for link in links:
    
    # scrap each product in home page list
    single_product = requests.get(link, headers = headers)
    soup = BeautifulSoup(single_product.text, 'html.parser')
    
    # scrap all products listed in a page
    products_ul = soup.find('ul', class_='inputlist clearfix')
    products = products_ul.find_all('a')

    # product headline
    headline = soup.find('h1', class_='primary product-item-headline').text


    for product in products:
        
        #product it
        sku = product.get('data-articlecode')
       
        # color
        color = product.get('data-color')
        
        # product id
        product_id = sku[:-3]
        
        # style id
        color_id = sku[-3:]

        # link
        link = 'https://www2.hm.com/en_us/productpage.{}.html'.format(sku)

        df_temp = pd.DataFrame( {'sku': sku, 'product_id' :product_id, 'color_id' : color_id, 'color': color, 'headline' : headline, 'link': link}, index = [0] )
        
        df_prods = pd.concat([df_prods, df_temp], axis = 0)


df_prods.drop_duplicates('sku',inplace = True)

df_prods.reset_index(inplace = True, drop = True)

In [27]:
df_prods.head()

,sku,product_id,color_id,color,headline,link
0,0427159001,0427159,001,Black denim,\n\t\t\t\t\t\t\t Trashed Skinny Jeans,https://www2.hm.com/en_us/productpage.04271590...
1,0427159002,0427159,002,Blue washed out,\n\t\t\t\t\t\t\t Trashed Skinny Jeans,https://www2.hm.com/en_us/productpage.04271590...
2,0427159003,0427159,003,Denim blue,\n\t\t\t\t\t\t\t Trashed Skinny Jeans,https://www2.hm.com/en_us/productpage.04271590...
3,0427159004,0427159,004,Light denim blue,\n\t\t\t\t\t\t\t Trashed Skinny Jeans,https://www2.hm.com/en_us/productpage.04271590...
4,0427159005,0427159,005,Dark denim blue,\n\t\t\t\t\t\t\t Trashed Skinny Jeans,https://www2.hm.com/en_us/productpage.04271590...


## Individual Scraping

### Scraping Data

In [6]:
# starting drive

options = Options()
options.headless = True
driver = webdriver.Firefox(options=options)

In [11]:
links[0]

'https://www2.hm.com/en_us/productpage.1024256001.html'

### Product Description

In [12]:
driver.get("https://www2.hm.com/en_us/productpage.0811993021.html")

# class_ = "BodyText-module--general__32l6J" # if below doesn't work
class_ = "ProductDescription-module--descriptionText__1zy9P"

try: 
    content = WebDriverWait(driver, 10).until(EC.presence_of_element_located( (By.CLASS_NAME, class_) ))
    desc = content.text
except:
    desc = 'NA'

content = WebDriverWait(driver, 10).until(EC.presence_of_element_located( (By.CLASS_NAME, class_) ))
desc = content.text
desc

'H&M Essentials. No. 2: The Jeans. 5-pocket jeans in stretch cotton denim. Regular waist, zip fly with button, and straight legs with good room for movement over thighs and knees.'

### Text

In [13]:
# driver.get(links[0])
driver.get("https://www2.hm.com/en_us/productpage.0811993021.html")

In [14]:
elements = driver.find_elements(by=By.CLASS_NAME, value="ProductAttributesList-module--descriptionListItem__3vUL2")
for e in elements:
    print(e.text)

FitRegular fit
CompositionCotton 98%, Spandex 2%
More sustainable materialsShell: Recycled cotton 21%
Art. No.0811993021


In [15]:
text = str()

text = [text + line.text  for line  in elements]
text

['FitRegular fit',
 'CompositionCotton 98%, Spandex 2%',
 'More sustainable materialsShell: Recycled cotton 21%',
 'Art. No.0811993021']

In [16]:
text_raw =' /'.join(text)
text_raw

'FitRegular fit /CompositionCotton 98%, Spandex 2% /More sustainable materialsShell: Recycled cotton 21% /Art. No.0811993021'

In [17]:
# searching for words fit and composition in all text retrieved from products web page
for element in text:
    if 'fit' in element:
        fit = element
    if 'Composition' in element:
        composition = element
print(fit)
print(composition)

FitRegular fit
CompositionCotton 98%, Spandex 2%


### Price

In [ ]:
https://www2.hm.com/en_us/productpage.1024256003.html

In [24]:
driver.get("https://www2.hm.com/en_us/productpage.0427159001.html")

class_price = "ProductPrice-module--productItemPrice__2i2Hc"
element = WebDriverWait(driver, 10).until( EC.presence_of_element_located( (By.CLASS_NAME, class_price) ) )
price = element.text

# if element returns empty, try this other class
if element.text == '':
    class_price = "price.parbase"
    element = WebDriverWait(driver, 10).until( EC.presence_of_element_located( (By.CLASS_NAME, class_price) ) )
    price = element.text
    
    if price == '':
        price = 'NA'
price

'$39.99'

### Joining Everything

In [17]:
# for idx, link in enumerate(df_prods['link'][-10:]):
#     print(link)

In [28]:
df_comp = pd.DataFrame()

for idx, link in enumerate(df_prods['link']):

    # sku
    sku = link.split('.')[3]
    # print('scraping page {}/{}: {}'.format( idx+1, len(df_prods), link))
    
    # load web page
    driver.get(link)
    
    # get price
    # try this class (for no promo days)
    class_price = "ProductPrice-module--productItemPrice__2i2Hc"
    element = WebDriverWait(driver, 10).until( EC.presence_of_element_located( (By.CLASS_NAME, class_price) ) )
    price = element.text

    # if element returns empty, try this other class
    if element.text == '':
        class_price = "price.parbase"
        element = WebDriverWait(driver, 10).until( EC.presence_of_element_located( (By.CLASS_NAME, class_price) ) )
        price = element.text
        
        if price == '':
            price = 'NA'
    
    # get product description   
    class_desc = "ProductDescription-module--descriptionText__1zy9P"      
    # test if description exists
    try: 
        content = WebDriverWait(driver, 10).until(EC.presence_of_element_located( (By.CLASS_NAME, class_desc) ))
        desc = content.text
    except:
        desc = 'NA'
    
    # get text
    class_text = 'ProductAttributesList-module--descriptionListItem__3vUL2'
    contents = WebDriverWait(driver, timeout=10).until( EC.presence_of_all_elements_located( (By.CLASS_NAME, class_text) ) )
    
    # concatenate all lines of text
    text = str()
    # list with all text
    text = [text + line.text  for line  in contents]

    # separate fit and composition from text
    # if fit or composition is not informed they'll return NA
    fit = 'NA'
    composition = 'NA'
    for element in text:
        if 'fit' in element:
            fit = element
        if 'Composition' in element:
            composition = element    
    
    # saving raw text
    text_raw =' /'.join(text)
    
    # saving results
    df_aux = pd.DataFrame( {'sku' : sku, 'price' : price, 'fit' : fit, 'composition' : composition, 'description' : desc ,'text' : text_raw,}, index = [0] )
    df_comp = pd.concat( [df_comp, df_aux], axis = 0 )     

df_comp.reset_index(inplace = True, drop = True)

KeyboardInterrupt: 

In [ ]:
df_comp.head()

,sku,price,fit,composition,description,text
0,1024256003,$19.99,FitSlim fit,"CompositionShell: Cotton 99%, Spandex 1%Pocket...",5-pocket jeans in stretch cotton denim. Regula...,"SizeThe model is 189cm/6'2"" and wears a size 3..."
1,1048642002,$39.99,FitRegular fit,"CompositionShell: Cotton 99%, Spandex 1%Pocket...",5-pocket jeans in stretch cotton denim with a ...,"FitRegular fit /CompositionShell: Cotton 99%, ..."


# Data Parsing

### Composition

In [ ]:
# removing composition using regex

df_comp_aux = df_comp.copy()

comps = []
linings = []

for idx, text in enumerate(df_comp_aux['composition']):
    # case 1 pocket lining present
    if 'Pocket' in text:
        # regex = '(Shell: .*?=Pocket|Cotton.*(?=Pocket))'
        regex = 'Cotton.*(?=Pocket)'
        try:
            comp = re.findall( regex, text)[0]
        except:
            comp = 'NA'
    # case 2 pocket lining not present
    else:
        regex = '(Cotton.*(?=Lining)|Cotton.*(?=lining)|Cotton.*%)'
        try:
            comp = re.findall( regex, text)[0]
        except:
            comp = 'NA'
        # print(df_comp_aux.loc[idx, 'sku'] + '|' + text +' | ' + comp)
    
    # geting pocket composition:
    regex = '(?<=lining: ).*'
    try:
        lining = re.findall(regex, text)[0]
    except:
        lining = 'Not Informed'
    linings.append(lining)
    
    comps.append(comp)
df_comp_aux['comp'] = comps
df_comp_aux['lining'] = linings

In [ ]:
# result
df_comp_aux.head()

,sku,price,fit,composition,description,text,comp,lining
0,1024256003,$19.99,FitSlim fit,"CompositionShell: Cotton 99%, Spandex 1%Pocket...",5-pocket jeans in stretch cotton denim. Regula...,"SizeThe model is 189cm/6'2"" and wears a size 3...","Cotton 99%, Spandex 1%","Polyester 65%, Cotton 35%"
1,1048642002,$39.99,FitRegular fit,"CompositionShell: Cotton 99%, Spandex 1%Pocket...",5-pocket jeans in stretch cotton denim with a ...,"FitRegular fit /CompositionShell: Cotton 99%, ...","Cotton 99%, Spandex 1%",Cotton 100%


In [ ]:
# creating a dataframe for all compositions
df_comp_split = pd.DataFrame()

for composition in df_comp_aux['comp']:
    comp_list = composition.split(' ') 

    # creating a df of compositions
    df_aux = composition_to_df(comp_list)

    # concatenating results
    df_comp_split = pd.concat( [df_comp_split, df_aux], axis = 0 )

df_comp_split.reset_index(inplace = True, drop = True)

In [ ]:
# result
df_comp_aux = pd.concat( [df_comp_aux, df_comp_split], axis = 1 )
df_comp_aux.head()


,sku,price,fit,composition,description,text,comp,lining,Cotton,Spandex
0,1024256003,$19.99,FitSlim fit,"CompositionShell: Cotton 99%, Spandex 1%Pocket...",5-pocket jeans in stretch cotton denim. Regula...,"SizeThe model is 189cm/6'2"" and wears a size 3...","Cotton 99%, Spandex 1%","Polyester 65%, Cotton 35%",99,1
1,1048642002,$39.99,FitRegular fit,"CompositionShell: Cotton 99%, Spandex 1%Pocket...",5-pocket jeans in stretch cotton denim with a ...,"FitRegular fit /CompositionShell: Cotton 99%, ...","Cotton 99%, Spandex 1%",Cotton 100%,99,1


In [ ]:
df_comp_aux.tail()

,sku,price,fit,composition,description,text,comp,lining,Cotton,Spandex
0,1024256003,$19.99,FitSlim fit,"CompositionShell: Cotton 99%, Spandex 1%Pocket...",5-pocket jeans in stretch cotton denim. Regula...,"SizeThe model is 189cm/6'2"" and wears a size 3...","Cotton 99%, Spandex 1%","Polyester 65%, Cotton 35%",99,1
1,1048642002,$39.99,FitRegular fit,"CompositionShell: Cotton 99%, Spandex 1%Pocket...",5-pocket jeans in stretch cotton denim with a ...,"FitRegular fit /CompositionShell: Cotton 99%, ...","Cotton 99%, Spandex 1%",Cotton 100%,99,1


### Fit

In [ ]:
# positive lookbehind + words I'm searching + positive lookahead
regex = "((?<=Fit).*(?= fit)|NA)"

df_comp_aux['fit'] = df_comp_aux['fit'].apply(lambda x: re.findall(regex, x)[0] )
df_comp_aux.head()

,sku,price,fit,composition,description,text,comp,lining,Cotton,Spandex
0,1024256003,$19.99,Slim,"CompositionShell: Cotton 99%, Spandex 1%Pocket...",5-pocket jeans in stretch cotton denim. Regula...,"SizeThe model is 189cm/6'2"" and wears a size 3...","Cotton 99%, Spandex 1%","Polyester 65%, Cotton 35%",99,1
1,1048642002,$39.99,Regular,"CompositionShell: Cotton 99%, Spandex 1%Pocket...",5-pocket jeans in stretch cotton denim with a ...,"FitRegular fit /CompositionShell: Cotton 99%, ...","Cotton 99%, Spandex 1%",Cotton 100%,99,1


### Price

In [37]:
# regex = "\$\d{2}.\d{2}\$\d{2}.\d{2}"
# price = '$22.22$11.11'
# price = '$31.99$39.99'

# bool(re.match(regex, price ))

In [ ]:
# df_comp_aux = df_comp_aux.copy()

# if there are 2 prices then there is a discount/promo
regex = "\$\d+\.\d+\$\d+.\d+"
df_comp_aux['isPromo'] = df_comp_aux['price'].apply(lambda x: 1 if bool(re.match(regex, x)) else 0)

# first price
regex = "^\$\d+\.\d+"
df_comp_aux['firstPrice'] = df_comp_aux['price'].apply( lambda x: re.findall(regex, x)[0] )

# second price
regex = "\$\d+\.\d+$"
df_comp_aux['secondPrice'] = df_comp_aux['price'].apply( lambda x: re.findall(regex, x)[0] )

# removing
df_comp_aux['firstPrice'] = df_comp_aux['firstPrice'].apply(lambda x: x.strip('$')).astype(float)
df_comp_aux['secondPrice'] = df_comp_aux['secondPrice'].apply(lambda x: x.strip('$')).astype(float)

# 
df_comp_aux['finalPrice'] = df_comp_aux.apply( lambda x: x['firstPrice'] if x['firstPrice'] <= x['secondPrice'] else x['secondPrice'], axis =1 )
df_comp_aux['originalPrice'] = df_comp_aux.apply( lambda x: x['secondPrice'] if x['secondPrice'] >= x['firstPrice'] else x['firstPrice'], axis =1 )

df_comp_aux.drop(['firstPrice', 'secondPrice'], axis = 1, inplace = True)

In [ ]:
# df_comp_aux[df_comp_aux.firstPrice == df_comp_aux.secondPrice]
df_comp_aux.head()

,sku,price,fit,composition,description,text,comp,lining,Cotton,Spandex,isPromo,finalPrice,originalPrice
0,1024256003,$19.99,Slim,"CompositionShell: Cotton 99%, Spandex 1%Pocket...",5-pocket jeans in stretch cotton denim. Regula...,"SizeThe model is 189cm/6'2"" and wears a size 3...","Cotton 99%, Spandex 1%","Polyester 65%, Cotton 35%",99,1,0,19.99,19.99
1,1048642002,$39.99,Regular,"CompositionShell: Cotton 99%, Spandex 1%Pocket...",5-pocket jeans in stretch cotton denim with a ...,"FitRegular fit /CompositionShell: Cotton 99%, ...","Cotton 99%, Spandex 1%",Cotton 100%,99,1,0,39.99,39.99


In [ ]:
df_comp_aux[df_comp_aux['isPromo'] == True].tail()

,sku,price,fit,composition,description,text,comp,lining,Cotton,Spandex,isPromo,finalPrice,originalPrice
8,0811993001,$29.99$12.99,Regular,"CompositionCotton 98%, Spandex 2%",H&M Essentials. No. 2: The Jeans. 5-pocket jea...,"FitRegular fit /CompositionCotton 98%, Spandex...","Cotton 98%, Spandex 2%",Not Informed,98,2,1,12.99,29.99


### Headline

In [ ]:
# removing whitespace characteres

df_prods['headline'] = df_prods['headline'].apply(lambda x: x.strip('\n\t ')) 
df_prods.headline.value_counts()

Slim Jeans                        33
Skinny Jeans                      26
Regular Jeans                     23
Trashed Skinny Jeans              15
Slim Tapered Jeans                14
Relaxed Jeans                     14
Hybrid Regular Tapered Joggers     7
Regular Tapered Crop Jeans         6
Freefit® Slim Jeans                5
Skinny Cropped Jeans               5
Loose Jeans                        4
Relaxed Tapered Pull-on Jeans      4
Hybrid Regular Denim Joggers       3
Regular Denim Joggers              3
Relaxed Pull-on Jeans              3
Relaxed Denim Joggers              2
Regular Bootcut Jeans              2
Cotton Denim Jeans                 1
Name: headline, dtype: int64

In [ ]:
df_comp_aux.head()


,sku,price,fit,composition,description,text,comp,lining,Cotton,Spandex,isPromo,finalPrice,originalPrice
0,1024256001,$19.99,Slim,"CompositionShell: Cotton 99%, Spandex 1%Pocket...",5-pocket jeans in stretch cotton denim. Regula...,"SizeThe model is 185cm/6'1"" and wears a size 3...","Cotton 99%, Spandex 1%","Polyester 65%, Cotton 35%",99,1,0,19.99,19.99


In [ ]:
df_final = pd.concat( [df_prods, df_comp_aux.drop('sku', axis = 1)], axis =1 )

# adding date time
now = datetime.now().strftime('%Y-%m-%d %H:%M:%S')
df_final['date'] = now
df_final.head()

NameError: name 'df_prods' is not defined

In [ ]:
df_final.columns

Index(['sku', 'product_id', 'color_id', 'color', 'headline', 'link', 'price',
       'fit', 'composition', 'description', 'text', 'comp', 'lining', 'Cotton',
       'Spandex', 'isPromo', 'finalPrice', 'originalPrice', 'date'],
      dtype='object')

In [147]:
#sku, product_id, color_id, color, fit, price, headline, 'cotton', 'polyester', 'elastane', 'elasterell_p', 'spandex' 'modal', 'viscose', pocket_lining, text

# selected_cols = ['sku', 'product_id', 'color_id', 'color', 'fit', 'finalPrice', 'originalPrice', "headline", 'Cotton', 'Polyester', 'Spandex', 'Modal', 'Elastomultiester', 'isPromo', 'description', 'text', 'link', 'date'] 
# rename_cols = ['sku', 'product_id', 'color_id', 'color', 'fit', 'final_price', 'original_price', "headline", 'cotton', 'polyester', 'spandex', 'modal', 'elastomultiester', 'is_promo', 'description', 'text', 'link', 'date'] 

# renaming some columns
selected_cols = ['finalPrice', 'originalPrice', 'isPromo'] 
rename_cols = ['final_price', 'original_price', 'is_promo'] 

final_cols = dict(zip(selected_cols, rename_cols))
df_final.rename(columns = final_cols, inplace = True )

# converting all columns to lower case
original_col = list(df_final.columns)
lower_col = [col.lower() for col in original_col]
final_cols = dict(zip(original_col, lower_col))

df_final.rename(columns = final_cols, inplace = True )

df_final.head()



,sku,product_id,color_id,color,headline,link,price,fit,composition,description,text,comp,lining,cotton,spandex,is_promo,final_price,original_price,date
0,1024256001,1024256,001,Black,Slim Jeans,https://www2.hm.com/en_us/productpage.10242560...,$19.99,Slim,"CompositionShell: Cotton 99%, Spandex 1%Pocket...",5-pocket jeans in stretch cotton denim. Regula...,"SizeThe model is 185cm/6'1"" and wears a size 3...","Cotton 99%, Spandex 1%","Polyester 65%, Cotton 35%",99,1,0.0,19.99,19.99,2022-04-18 18:25:55
1,1024256002,1024256,002,Light denim blue,Slim Jeans,https://www2.hm.com/en_us/productpage.10242560...,$19.99,Slim,"CompositionShell: Cotton 99%, Spandex 1%Pocket...",5-pocket jeans in stretch cotton denim. Regula...,"SizeThe model is 189cm/6'2"" and wears a size 3...","Cotton 99%, Spandex 1%","Polyester 65%, Cotton 35%",99,1,0.0,19.99,19.99,2022-04-18 18:25:55
2,1024256003,1024256,003,Light denim blue,Slim Jeans,https://www2.hm.com/en_us/productpage.10242560...,$19.99,Slim,"CompositionShell: Cotton 99%, Spandex 1%Pocket...",5-pocket jeans in stretch cotton denim. Regula...,"SizeThe model is 189cm/6'2"" and wears a size 3...","Cotton 99%, Spandex 1%","Polyester 65%, Cotton 35%",99,1,0.0,19.99,19.99,2022-04-18 18:25:55
3,1024256004,1024256,004,Denim blue,Slim Jeans,https://www2.hm.com/en_us/productpage.10242560...,$19.99,Slim,"CompositionShell: Cotton 99%, Spandex 1%Pocket...",5-pocket jeans in stretch cotton denim. Regula...,"FitSlim fit /CompositionShell: Cotton 99%, Spa...","Cotton 99%, Spandex 1%","Polyester 65%, Cotton 35%",99,1,0.0,19.99,19.99,2022-04-18 18:25:55
4,1024256005,1024256,005,Dark blue,Slim Jeans,https://www2.hm.com/en_us/productpage.10242560...,$19.99,Slim,"CompositionShell: Cotton 99%, Spandex 1%Pocket...",5-pocket jeans in stretch cotton denim. Regula...,"SizeThe model is 180cm/5'11"" and wears a size ...","Cotton 99%, Spandex 1%","Polyester 65%, Cotton 35%",99,1,0.0,19.99,19.99,2022-04-18 18:25:55


### Converting Data Types

In [148]:
# converting dtypes to numeric
df_final.dtypes

sku                object
product_id         object
color_id           object
color              object
headline           object
link               object
price              object
fit                object
composition        object
description        object
text               object
comp               object
lining             object
cotton             object
spandex            object
is_promo          float64
final_price       float64
original_price    float64
date               object
dtype: object

In [149]:
# fill NAs
df_final.isna().sum()


sku                 0
product_id          0
color_id            0
color               0
headline            0
link                0
price             160
fit               160
composition       160
description       160
text              160
comp              160
lining            160
cotton            160
spandex           160
is_promo          160
final_price       160
original_price    160
date                0
dtype: int64

In [150]:
df_final.fillna(0, inplace = True)


In [151]:
# selects all numerical columns
cols_to_num = list(df_final.columns)
str_cols = ['sku','product_id','color_id', 'color', 'fit', 'price', 'final_price', 'original_price', 'headline', 'description', 'composition', 'comp', 'lining', 'text', 'link', 'date']
for col in str_cols:
    cols_to_num.remove(col)

cols_to_num
print(cols_to_num)

for col in cols_to_num:
    try:
        # convert to float then to int (to avoid NA to int error)
        df_final[col] = df_final[col].astype(int)
    except:
        df_final[col] = df_final[col].astype('Int64')
    finally:
        pass

# converting date to datetime
df_final['date'] = pd.to_datetime( df_final['date'], errors = 'coerce')

['cotton', 'spandex', 'is_promo']


In [152]:
df_final.dtypes

sku                       object
product_id                object
color_id                  object
color                     object
headline                  object
link                      object
price                     object
fit                       object
composition               object
description               object
text                      object
comp                      object
lining                    object
cotton                     int64
spandex                    int64
is_promo                   int64
final_price              float64
original_price           float64
date              datetime64[ns]
dtype: object

In [ ]:
# changing columns order to match db standard

'sku',
'product_id',
'color_id',
'color',
'fit',
'final_price',
'original_price',
'headline',
#'cotton',
#'spandex',
'is_promo',
'description',
'text',
'link',
'date',

'price', # Remove
'composition', #remove
'comp', # remove
'lining', # add

In [153]:
df_final.head()

,sku,product_id,color_id,color,headline,link,price,fit,composition,description,text,comp,lining,cotton,spandex,is_promo,final_price,original_price,date
0,1024256001,1024256,001,Black,Slim Jeans,https://www2.hm.com/en_us/productpage.10242560...,$19.99,Slim,"CompositionShell: Cotton 99%, Spandex 1%Pocket...",5-pocket jeans in stretch cotton denim. Regula...,"SizeThe model is 185cm/6'1"" and wears a size 3...","Cotton 99%, Spandex 1%","Polyester 65%, Cotton 35%",99,1,0,19.99,19.99,2022-04-18 18:25:55
1,1024256002,1024256,002,Light denim blue,Slim Jeans,https://www2.hm.com/en_us/productpage.10242560...,$19.99,Slim,"CompositionShell: Cotton 99%, Spandex 1%Pocket...",5-pocket jeans in stretch cotton denim. Regula...,"SizeThe model is 189cm/6'2"" and wears a size 3...","Cotton 99%, Spandex 1%","Polyester 65%, Cotton 35%",99,1,0,19.99,19.99,2022-04-18 18:25:55
2,1024256003,1024256,003,Light denim blue,Slim Jeans,https://www2.hm.com/en_us/productpage.10242560...,$19.99,Slim,"CompositionShell: Cotton 99%, Spandex 1%Pocket...",5-pocket jeans in stretch cotton denim. Regula...,"SizeThe model is 189cm/6'2"" and wears a size 3...","Cotton 99%, Spandex 1%","Polyester 65%, Cotton 35%",99,1,0,19.99,19.99,2022-04-18 18:25:55
3,1024256004,1024256,004,Denim blue,Slim Jeans,https://www2.hm.com/en_us/productpage.10242560...,$19.99,Slim,"CompositionShell: Cotton 99%, Spandex 1%Pocket...",5-pocket jeans in stretch cotton denim. Regula...,"FitSlim fit /CompositionShell: Cotton 99%, Spa...","Cotton 99%, Spandex 1%","Polyester 65%, Cotton 35%",99,1,0,19.99,19.99,2022-04-18 18:25:55
4,1024256005,1024256,005,Dark blue,Slim Jeans,https://www2.hm.com/en_us/productpage.10242560...,$19.99,Slim,"CompositionShell: Cotton 99%, Spandex 1%Pocket...",5-pocket jeans in stretch cotton denim. Regula...,"SizeThe model is 180cm/5'11"" and wears a size ...","Cotton 99%, Spandex 1%","Polyester 65%, Cotton 35%",99,1,0,19.99,19.99,2022-04-18 18:25:55


# Data Saving

In [ ]:
# df_final.to_sql('hm_showroom', con = )D

In [46]:
# Saving Locally
now = datetime.now().strftime('%Y-%m-%d %H:%M:%S')

# df_comp.to_csv(path_or_buf='./backups/df_comp-{}.csv'.format(now), )

# saving df as a local backup
# df_final.to_csv('../backups/df_backup-{}.csv'.format(now), index = False)

print('saved: df_backup-{}.csv'.format(now))

saved: df_backup-2022-04-18 16:31:58.csv


In [137]:
df_final.dtypes

sku                       object
product_id                object
color_id                  object
color                     object
headline                  object
link                      object
price                     object
fit                       object
composition               object
description               object
text                      object
comp                      object
lining                    object
cotton                     int64
spandex                    int64
is_promo                   int64
final_price              float64
original_price           float64
date              datetime64[ns]
dtype: object

In [138]:
# Inserting processed data into MySQL DB
# path = 'sqlite:///' + path_to_db
path = 'sqlite:///' + '/home/humberto/DS/hm/jeans_db.sqlite'
# creating sqlalchemy engine for connection
engine = create_engine(path, echo=True)

# creating a Session class
Session = sessionmaker(bind=engine)

# creating a session
session = Session()
# testing case a new column is added

In [139]:
try:
    # adding data
    df_final.to_sql('hm_showroom', con = engine, if_exists='append', index = False)

    # committing changes
    session.commit()
except:
    try:
        # in case scraped data returns with a new column, it will be added to a new table
        table_name = "hm_showroom_backup-{}".format(datetime.now().strftime("%Y-%m-%d"))
        df_final.to_sql( table_name, con = engine, if_exists='append', index = False)

        session.commit()
    except:  
        # if even this fails, undo everything      
        session.rollback()

finally:
    session.close()

# USE THIS SCRIPT TO PREVENT FAILS

2022-04-18 18:19:34,673 INFO sqlalchemy.engine.Engine PRAGMA main.table_info("hm_showroom")
2022-04-18 18:19:34,676 INFO sqlalchemy.engine.Engine [raw sql] ()
2022-04-18 18:19:34,733 INFO sqlalchemy.engine.Engine BEGIN (implicit)
2022-04-18 18:19:34,748 INFO sqlalchemy.engine.Engine INSERT INTO hm_showroom (sku, product_id, color_id, color, headline, link, price, fit, composition, description, text, comp, lining, cotton, spandex, is_promo, final_price, original_price, date) VALUES (?, ?, ?, ?, ?, ?, ?, ?, ?, ?, ?, ?, ?, ?, ?, ?, ?, ?, ?)
2022-04-18 18:19:34,751 INFO sqlalchemy.engine.Engine [generated in 0.00974s] (('1024256001', '1024256', '001', 'Black', 'Slim Jeans', 'https://www2.hm.com/en_us/productpage.1024256001.html', '$19.99', 'Slim', 'CompositionShell: Cotton 99%, Spandex 1%Pocket lining: Polyester 65%, Cotton 35%', '5-pocket jeans in stretch cotton denim. Regular waist, zip fly with button, and slim legs.', 'SizeThe model is 185cm/6\'1" and wears a size 31/32 /FitSlim fit /C

In [113]:
df_old.columns

Index(['sku', 'product_id', 'color_id', 'color', 'fit', 'final_price',
       'Composition', 'text', 'cotton', 'polyester', 'elastane',
       'elasterell_p', 'modal', 'viscose', 'link', 'date', 'lining',
       'headline', 'original_price', 'is_promo', 'description'],
      dtype='object')

### Merging all data and inserting it to MySQL on AWS

In [27]:
# # creating sqlalchemy engine for connection
# path = 'sqlite:///' + path_to_db
# engine = create_engine(path, echo=False)

# # creating a Session class
# Session = sessionmaker(bind=engine)

# # creating a session
# session = Session()
# query = """
# DROP TABLE IF EXISTS hm_showroom
# """

# try:
#     engine.execute(query)
#     session.commit()
# except:
#     session.rollback()

In [127]:
df_old.isna().sum()
df_old.shape

(5390, 19)

In [143]:
# filling NAs
df_final_13_04.head()
df_final_13_04.isna().sum()
df_final_13_04[df_final_13_04['fit'].isna()]

# # filling NAs
df_final_13_04.loc[df_final_13_04['fit'].isna()] = 'Not Informed'
df_final_13_04.fillna(value = 0, inplace = True)
df_final_13_04.isna().sum()


sku                 0
product_id          0
color_id            0
color               0
fit                 0
final_price         0
original_price      0
headline            0
cotton              0
polyester           0
spandex             0
modal               0
elastomultiester    0
is_promo            0
description         0
text                0
link                0
date                0
dtype: int64

In [144]:
df_final_13_04.head()

,sku,product_id,color_id,color,fit,final_price,original_price,headline,cotton,polyester,spandex,modal,elastomultiester,is_promo,description,text,link,date
0,0427159001,0427159,001,Black denim,Skinny,39.99,39.99,Trashed Skinny Jeans,91,7.0,2,0,0,0,5-pocket jeans in washed stretch denim. Heavil...,"FitSkinny fit /CompositionCotton 91%, Polyeste...",https://www2.hm.com/en_us/productpage.04271590...,2022-04-13 21:49:54
1,0427159002,0427159,002,Blue washed out,Skinny,39.99,39.99,Trashed Skinny Jeans,93,6.0,1,0,0,0,5-pocket jeans in washed stretch denim. Heavil...,"FitSkinny fit /CompositionCotton 93%, Polyeste...",https://www2.hm.com/en_us/productpage.04271590...,2022-04-13 21:49:54
2,0427159003,0427159,003,Denim blue,Skinny,39.99,39.99,Trashed Skinny Jeans,93,6.0,1,0,0,0,5-pocket jeans in washed stretch denim. Heavil...,"FitSkinny fit /CompositionCotton 93%, Polyeste...",https://www2.hm.com/en_us/productpage.04271590...,2022-04-13 21:49:54
3,0427159004,0427159,004,Light denim blue,Skinny,39.99,39.99,Trashed Skinny Jeans,99,0,1,0,0,0,5-pocket jeans in washed stretch denim. Heavil...,"FitSkinny fit /CompositionCotton 99%, Spandex ...",https://www2.hm.com/en_us/productpage.04271590...,2022-04-13 21:49:54
4,0427159005,0427159,005,Dark denim blue,Skinny,39.99,39.99,Trashed Skinny Jeans,72,20.0,1,7.0,0,0,5-pocket jeans in washed stretch denim. Heavil...,"FitSkinny fit /CompositionCotton 72%, Polyeste...",https://www2.hm.com/en_us/productpage.04271590...,2022-04-13 21:49:54


In [145]:
df_final_16_04.isna().sum()
df_final_16_04[df_final_16_04['fit'].isna()]

# # filling NAs
df_final_16_04.loc[df_final_16_04['fit'].isna()] = 'Not Informed'
df_final_16_04.fillna(value = 0, inplace = True)
df_final_16_04.isna().sum()

sku                 0
product_id          0
color_id            0
color               0
fit                 0
final_price         0
original_price      0
headline            0
cotton              0
polyester           0
spandex             0
modal               0
elastomultiester    0
is_promo            0
description         0
text                0
link                0
date                0
dtype: int64

In [146]:
df_old.columns

Index(['sku', 'product_id', 'color_id', 'color', 'fit', 'final_price',
       'original_price', 'headline', 'cotton', 'polyester', 'elastane',
       'elasterell_p', 'viscose', 'modal', 'is_promo', 'description', 'text',
       'link', 'date'],
      dtype='object')

In [147]:
df_final.columns

Index(['sku', 'product_id', 'color_id', 'color', 'fit', 'final_price',
       'original_price', 'headline', 'cotton', 'polyester', 'spandex', 'modal',
       'elastomultiester', 'is_promo', 'description', 'text', 'link', 'date'],
      dtype='object')

In [148]:
df_merged = pd.concat([df_old, df_final_13_04, df_final_16_04], axis = 0)
df_merged.head()

# changing columns order:
cols = ['sku','product_id','color_id','color','fit','final_price','original_price','headline','cotton','polyester', 'elastane', 'spandex','elastomultiester', 'elasterell_p','viscose','modal','is_promo','description','text','link','date']
df_merged = df_merged[cols].copy()

df_merged.isna().sum()

sku                    0
product_id             0
color_id               0
color                  0
fit                    0
final_price            0
original_price         0
headline               0
cotton                 0
polyester              0
elastane             348
spandex             5390
elastomultiester    5390
elasterell_p         348
viscose              348
modal                  0
is_promo               0
description            0
text                   0
link                   0
date                   0
dtype: int64

In [149]:
# filling NAs
df_merged.fillna(value = 0, inplace = True)

df_merged.isna().sum()

sku                 0
product_id          0
color_id            0
color               0
fit                 0
final_price         0
original_price      0
headline            0
cotton              0
polyester           0
elastane            0
spandex             0
elastomultiester    0
elasterell_p        0
viscose             0
modal               0
is_promo            0
description         0
text                0
link                0
date                0
dtype: int64

In [150]:
df_merged.head()


,sku,product_id,color_id,color,fit,final_price,original_price,headline,cotton,polyester,...,spandex,elastomultiester,elasterell_p,viscose,modal,is_promo,description,text,link,date
0,0690449001,0690449,001,Light denim blue/trashed,Skinny,16.99,16.99,Light denim blue/trashed,99,0,...,0,0,0.0,0.0,0,0,Not Informed,"Cotton 99%, Elastane 1%",https://www2.hm.com/en_us/productpage.06904490...,2021-09-11 17:19:36
1,0690449002,0690449,002,Denim blue,Skinny,14.99,14.99,Denim blue,98,0,...,0,0,0.0,0.0,0,0,Not Informed,"Cotton 98%, Elastane 2%",https://www2.hm.com/en_us/productpage.06904490...,2021-09-11 17:19:36
2,0690449006,0690449,006,Black/washed,Skinny,7.99,7.99,Black/washed,100,0,...,0,0,0.0,0.0,0,0,Not Informed,"Cotton 100% Cotton 98%, Elastane 2%",https://www2.hm.com/en_us/productpage.06904490...,2021-09-11 17:19:36
3,0690449007,0690449,007,Light denim blue,Skinny,14.99,14.99,Light denim blue,98,0,...,0,0,0.0,0.0,0,0,Not Informed,"Cotton 98%, Elastane 2% Lining",https://www2.hm.com/en_us/productpage.06904490...,2021-09-11 17:19:36
4,0690449009,0690449,009,Black washed out,Skinny,19.99,19.99,Black washed out,99,0,...,0,0,0.0,0.0,0,0,Not Informed,"Cotton 99%, Elastane 1%",https://www2.hm.com/en_us/productpage.06904490...,2021-09-11 17:19:36


In [151]:
df_merged.shape

(5738, 21)

In [152]:
df_merged.spandex.unique()


array([0, 2, 1, 'Not Informed', '2', '1'], dtype=object)

In [154]:
# removing bug
df_merged.spandex.unique()
df_merged[df_merged.spandex == 'Not Informed']
df_merged = df_merged[ df_merged['spandex'] != 'Not Informed'].copy()


In [155]:
# converting dtypes
df_merged['final_price'] = df_merged['final_price'].astype(float)
df_merged['original_price'] = df_merged['original_price'].astype(float)
df_merged['cotton'] = df_merged['cotton'].astype(int)
df_merged['polyester'] = df_merged['polyester'].astype(int)
df_merged['elastane'] = df_merged['elastane'].astype(int)
df_merged['spandex'] = df_merged['spandex'].astype(int)
df_merged['elastomultiester'] = df_merged['elastomultiester'].astype(int)
df_merged['elasterell_p'] = df_merged['elasterell_p'].astype(int)
df_merged['viscose'] = df_merged['viscose'].astype(int)
df_merged['modal'] = df_merged['modal'].astype(int)
df_merged['is_promo'] = df_merged['is_promo'].astype(int)
df_merged['date'] = pd.to_datetime(df_merged['date'], errors = 'coerce')

In [167]:
# checking if there are no bugs or errors
df_merged.cotton.unique()
df_merged['polyester'].unique()
df_merged['elastane'].unique()
df_merged['spandex' ].unique()
df_merged['elastomultiester' ].unique()
df_merged['elasterell_p' ].unique()
df_merged['viscose' ].unique()
df_merged['modal'].unique()
df_merged['is_promo'].unique()

array([0, 1])

In [168]:
df_merged.dtypes

sku                         object
product_id                  object
color_id                    object
color                       object
fit                         object
final_price                float64
original_price             float64
headline                    object
cotton                       int64
polyester                    int64
elastane                     int64
spandex                      int64
elastomultiester             int64
elasterell_p                 int64
viscose                      int64
modal                        int64
is_promo                     int64
description                 object
text                        object
link                        object
date                datetime64[ns]
dtype: object

In [176]:
df_merged[df_merged['original_price'] < df_merged['final_price'] ].shape

(0, 21)

In [177]:
df_merged.tail()

,sku,product_id,color_id,color,fit,final_price,original_price,headline,cotton,polyester,...,spandex,elastomultiester,elasterell_p,viscose,modal,is_promo,description,text,link,date
166,1027852007,1027852,007,Denim blue,Relaxed,23.99,29.99,Relaxed Denim Joggers,100,0,...,0,0,0,0,0,1,"Joggers in thick cotton denim. Regular waist, ...","SizeThe model is 185cm/6'1"" and wears a size M...",https://www2.hm.com/en_us/productpage.10278520...,2022-04-16 19:13:21
167,1048642001,1048642,001,Denim blue,Regular,31.99,39.99,Regular Bootcut Jeans,99,0,...,1,0,0,0,0,1,5-pocket jeans in stretch cotton denim with a ...,"FitRegular fit /CompositionShell: Cotton 99%, ...",https://www2.hm.com/en_us/productpage.10486420...,2022-04-16 19:13:21
168,1048642002,1048642,002,Denim black,Regular,31.99,39.99,Regular Bootcut Jeans,99,0,...,1,0,0,0,0,1,5-pocket jeans in stretch cotton denim with a ...,"FitRegular fit /CompositionShell: Cotton 99%, ...",https://www2.hm.com/en_us/productpage.10486420...,2022-04-16 19:13:21
169,1049466001,1049466,001,Denim blue,Loose,31.99,39.99,Loose Carpenter Jeans,100,0,...,0,0,0,0,0,1,"Jeans in thick cotton denim. Regular waist, zi...",FitLoose fit /CompositionCotton 100% /More sus...,https://www2.hm.com/en_us/productpage.10494660...,2022-04-16 19:13:21
170,1063426001,1063426,001,Light indigo blue,NA,39.99,49.99,Cotton Denim Jeans,100,0,...,0,0,0,0,0,1,Edition by is an uncompromising collection of ...,"SizeThe model is 188cm/6'2"" and wears a size 3...",https://www2.hm.com/en_us/productpage.10634260...,2022-04-16 19:13:21


In [181]:
# reading credentials

secrets_json = open('../secrets/secrets.json')
secrets = json.load(secrets_json)

dialect =   secrets["dialect"]
driver =    secrets["driver"]
host =      secrets["host"]
username =  secrets["username"]
password =  secrets["password"]
port =      secrets["port"]
database =  secrets["database"]

url = "{}+{}://{}:{}@{}:{}/{}".format(dialect, driver, username, password, host, port, database)
# engine = create_engine(url = url, echo = True) #, pool_pre_ping = True

In [182]:
# Inserting processed data into MySQL DB

# creating sqlalchemy engine for connection
engine = create_engine(url, echo=True)

# creating a Session class
Session = sessionmaker(bind=engine)

# creating a session
session = Session()

In [186]:
# testing case a new column is added

try:
    # adding data
    df_merged.to_sql('hm_showroom', con = engine, if_exists='append', index = False)

    # committing changes
    session.commit()
except:
    try:
        # in case scraped data returns with a new column, it will be added to a new table
        table_name = "hm_showroom_backup-{}".format(datetime.now().strftime("%Y-%m-%d"))
        df_merged.to_sql( table_name, con = engine, if_exists='append', index = False)

        session.commit()
    except:  
        # if even this fails, undo everything      
        session.rollback()

finally:
    session.close()

# USE THIS SCRIPT TO PREVENT FAILS

2022-04-16 21:21:59,065 INFO sqlalchemy.engine.Engine SELECT COUNT(*) FROM information_schema.tables WHERE table_schema = %(table_schema)s AND table_name = %(table_name)s
2022-04-16 21:21:59,066 INFO sqlalchemy.engine.Engine [cached since 278.7s ago] {'table_schema': 'humberto_personal_projects', 'table_name': 'hm_showroom'}
2022-04-16 21:21:59,116 INFO sqlalchemy.engine.Engine BEGIN (implicit)
2022-04-16 21:21:59,119 INFO sqlalchemy.engine.Engine 
CREATE TABLE hm_showroom (
	sku TEXT, 
	product_id TEXT, 
	color_id TEXT, 
	color TEXT, 
	fit TEXT, 
	final_price FLOAT(53), 
	original_price FLOAT(53), 
	headline TEXT, 
	cotton BIGINT, 
	polyester BIGINT, 
	elastane BIGINT, 
	spandex BIGINT, 
	elastomultiester BIGINT, 
	elasterell_p BIGINT, 
	viscose BIGINT, 
	modal BIGINT, 
	is_promo BIGINT, 
	description TEXT, 
	text TEXT, 
	link TEXT, 
	date DATETIME
)


2022-04-16 21:21:59,122 INFO sqlalchemy.engine.Engine [no key 0.00325s] {}
2022-04-16 21:21:59,168 INFO sqlalchemy.engine.Engine COMMI

In [189]:
session.close_all()

/tmp/ipykernel_7084/2483070458.py:1: SADeprecationWarning: The Session.close_all() method is deprecated and will be removed in a future release.  Please refer to session.close_all_sessions(). (deprecated since: 1.3)
  session.close_all()


## TODO

- clean script
- check datatypes and use datetime for date column DONE
- automate data insertion
- create db on AWS DONE
- create EC2 to run script
- create Lambda with cronjob to run script